In [2]:
import pandas as pd
import mysql.connector as mariadb
from pymongo import MongoClient
from sqlalchemy import create_engine
from config.constants import LEAGUES_DATA_DICT, CSV_EXPORT_PATH, CSV_EXPORT_PATH_MERGED
from converters.data2frames import read_json
import os
from tqdm import tqdm
import numpy as np

#  MongoDB

In [77]:
client2 = MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [80]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [81]:
db = client2.slds

In [82]:
m_coll = db.soloq_m
tl_coll = db.soloq_tl

In [7]:
def __get_file_names_from_match_id(m_id, save_dir):
    file_names = os.listdir(save_dir)
    match_filename = [val for val in file_names if str(m_id) in val and 'tl' not in val][0]
    tl_filename = [val for val in file_names if str(m_id) in val and 'tl' in val][0]
    return {'match_filename': match_filename.split('.')[0], 'tl_filename': tl_filename.split('.')[0]}

In [9]:
LEAGUES_DATA_DICT['SOLOQ']

{'csv_export_path': '../exports/soloq_dataset.csv',
 'csv_export_path_merged': '../exports/soloq_dataset_merged.csv',
 'dtypes': {},
 'excel_export_path': '../exports/soloq_dataset.xlsx',
 'excel_export_path_merged': '../exports/soloq_dataset_merged.xlsx',
 'ids_file_path': '../leagues_data/soloq.csv',
 'official_league': False,
 'raw_data_path': '../matches_raw_data/soloq/'}

In [21]:
read_dir = LEAGUES_DATA_DICT['SOLOQ']['raw_data_path']
f_names = os.listdir(read_dir)
m_ids = [name.split('.')[0].split('_')[1] for name in f_names if 'tl' not in name]

In [50]:
for g_id in tqdm(m_ids):
    f1 = read_json(save_dir=read_dir, file_name=__get_file_names_from_match_id(m_id=g_id, save_dir=read_dir)['tl_filename'])
    f1['gameId'] = int(g_id)
    tl_coll.insert_one(f1)

100%|████████████████████████████████████████████████████████████████████████████| 14662/14662 [24:05<00:00, 10.08it/s]


In [83]:
cursor = m_coll.find({}, {'_id': 0, 'gameId': 1})

In [84]:
l1 = [gid['gameId'] for gid in cursor]

In [85]:
len(l1)

14662

# MariaDB

In [14]:
df = pd.read_excel(LEAGUES_DATA_DICT['SOLOQ']['excel_export_path_merged']).drop('summonerName', axis=1)

KeyboardInterrupt: 

In [23]:
df2 = df.drop('summonerName', axis=1) 

In [86]:
engine = create_engine('mysql+mysqlconnector://davidherasp:.@127.0.0.1/exports')

In [25]:
id_list = [num for num in df2.gameId.unique()]

In [26]:
for num in id_list:
    df3 = df2.loc[df.gameId == num]
    df3.to_sql('soloq', engine, if_exists='append')

In [28]:
cnx = mariadb.connect(user='davidherasp', database='exports', password='.')

In [95]:
SQL_DEFAULT_CONN = {
    'user': 'davidherasp',
    'password': '.',
    'host': '127.0.0.1',
    'database': '',
    'raise_on_warnings': True,
    'use_pure': False,
}

In [96]:
SQL1 = SQL_DEFAULT_CONN
SQL1['database'] = 'exports' 

In [97]:
SQL_DEFAULT_CONN

{'database': 'exports',
 'host': '127.0.0.1',
 'password': '.',
 'raise_on_warnings': True,
 'use_pure': False,
 'user': 'davidherasp'}

In [29]:
cursor = cnx.cursor()

In [48]:
query = ('SELECT DISTINCT gameId FROM soloq')

In [54]:
cursor.execute(query)

In [57]:
l1 = [gid[0] for gid in cursor]

In [6]:
df = pd.read_csv(LEAGUES_DATA_DICT['SOLOQ'][CSV_EXPORT_PATH_MERGED], index_col=0, encoding='ISO-8859-1')

C:\Users\david\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (47,48,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.loc[df.team_abbv.isin(['NIP'])].to_csv('mrs_soloq.csv')

In [8]:
df.loc[df.team_abbv.isin(['NIP'])].to_excel('nip_soloq.xlsx')